In [7]:
from tensorflow.keras.models import load_model

# Load your trained CNN model
cnn_model = load_model('../../models/hybrid_skin_disease_model.keras')


In [8]:
from tensorflow.keras.models import Model

# Select the layer where you want to stop (e.g., second last layer)
feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator for preprocessing
datagen = ImageDataGenerator(rescale=1.0/255.0)  # Normalize pixel values (0-255) to (0-1)
train_dir = "../../data2/train"
validation_dir = "../../data2/validation"
test_dir = "../../data2/test"
# Create data generators
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to match the CNN input
    batch_size=32,
    class_mode='categorical',  # For multi-class classification
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Keep order for consistent evaluation
)


Found 4000 images belonging to 4 classes.
Found 1200 images belonging to 4 classes.
Found 1200 images belonging to 4 classes.


In [10]:
# Get the next batch of images and labels from the train generator
images, labels = next(train_generator)

# Or loop through the generator to process all batches
# for batch_images, batch_labels in train_generator:
    # print(batch_images.shape, batch_labels.shape)
    # Do something with the batch


In [11]:
import numpy as np

# Function to extract features
def extract_features(generator, feature_extractor):
    features = []
    labels = []
    
    for batch_images, batch_labels in generator:
        batch_features = feature_extractor.predict(batch_images)  # Use your CNN feature extractor
        features.append(batch_features)
        labels.append(batch_labels)
        
        # Break the loop when all data has been processed
        if generator.batch_index == 0:  # At the end of one epoch
            break
    
    # Concatenate batches
    return np.vstack(features), np.vstack(labels)

# Assuming 'feature_extractor' is your modified CNN model
train_features, train_labels = extract_features(train_generator, feature_extractor)
validation_features, validation_labels = extract_features(validation_generator, feature_extractor)
test_features, test_labels = extract_features(test_generator, feature_extractor)

# Save features for future use
np.save('train_features.npy', train_features)
np.save('train_labels.npy', train_labels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split

# Load extracted features and corresponding labels
features = np.load('train_features.npy')  # Feature vectors from the CNN
labels = np.load('train_labels.npy')      # Labels for your data

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


In [13]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # Adjust 'n_estimators' as needed
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the RF model
print("Random Forest Classifier Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.2f}")
print(classification_report(y_test, y_pred_rf))


Random Forest Classifier Performance:


NameError: name 'accuracy_score' is not defined

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train KNN
knn_model = KNeighborsClassifier(n_neighbors=5)  # Adjust 'n_neighbors' as needed
knn_model.fit(X_train, y_train)

# Predict on the test set
y_pred_knn = knn_model.predict(X_test)

# Evaluate the KNN model
print("KNN Classifier Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")
print(classification_report(y_test, y_pred_knn))
